In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image, ImageOps
import numpy as np
import cv2
import itertools

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# -------- Definizione del modello ResNet18 modificato a 6 canali --------
from torchvision.models import resnet18

class ResNetRegressor(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model
    
    def forward(self, x):
        x = self.base(x)
        x = torch.sigmoid(x)  # output tra 0 e 1
        return x

# Carica modello
base_model = resnet18(pretrained=False)
base_model.conv1 = nn.Conv2d(6, 64, kernel_size=7, stride=2, padding=3, bias=False)
base_model.fc = nn.Linear(base_model.fc.in_features, 2)  # output 2 valori per regressione
model = ResNetRegressor(base_model).to(device)

# Carica pesi salvati
model.load_state_dict(torch.load("regression_resnet18_finetuned.pth", map_location=device))
model.eval()

# -------- Custom Transform: CenterPadCrop --------
class CenterPadCrop:
    """Ridimensiona mantenendo aspect ratio e aggiunge padding centrato per ottenere quadrato finale"""
    def __init__(self, final_size=256):
        self.final_size = final_size

    def __call__(self, img: Image.Image):
        # --- Resize lato lungo a final_size ---
        w, h = img.size
        if h > w:
            new_h = self.final_size
            new_w = int(w * self.final_size / h)
        else:
            new_w = self.final_size
            new_h = int(h * self.final_size / w)
        img = img.resize((new_w, new_h), resample=Image.BILINEAR)

        # --- Calcola padding per rendere quadrato centrato ---
        pad_left = (self.final_size - new_w) // 2
        pad_right = self.final_size - new_w - pad_left
        pad_top = (self.final_size - new_h) // 2
        pad_bottom = self.final_size - new_h - pad_top

        # --- Applica padding ---
        img = transforms.functional.pad(img, padding=(pad_left, pad_top, pad_right, pad_bottom), fill=0)

        return img

# -------- Trasformazioni immagini --------
transform_rgb = transforms.Compose([
    CenterPadCrop(final_size=256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

edge_transform = transforms.Compose([
    CenterPadCrop(final_size=256),
    transforms.ToTensor(),
])

# -------- Funzione per leggere immagine + edge e concatenare --------
def load_wheel_image(path):
    img = cv2.imread(path) # legge l'immagine
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # converte BGR a RGB, fondamentale per i modelli torch

    # Edge detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # converte a scala di grigi
    #gray_rgb = np.stack([gray]*3, axis=-1)
    edges = cv2.Canny(gray, 20, 60) # rilevamento bordi con Canny
    edges_rgb = np.stack([edges]*3, axis=-1)  # convert to 3 channels per compatibilità con edge_transform

    img = Image.fromarray(img) # converte a PIL Image
    img = ImageOps.exif_transpose(img)
    #gray_rgb = Image.fromarray(gray_rgb)
    edges_rgb = Image.fromarray(edges_rgb)
    edges_rgb = ImageOps.exif_transpose(edges_rgb)

    img = transform_rgb(img)
    edges_rgb = edge_transform(edges_rgb)

    # Concatenate RGB image and edge image along channel dimension
    combined = torch.cat((img, edges_rgb), dim=0) # concatenazione lungo la dimensione dei canali, avremo 6 canali

    return combined

# -------- Lista di 8 ruote da processare --------
wheel_paths = [
    "ruota1.jpeg",
    "ruota2.jpeg",
    "ruota3.jpeg",
    "ruota4.jpeg",
    "ruota5.jpeg",
    "ruota6.jpeg",
    "ruota7.jpeg",
    "ruota8.jpeg"
]

positions_mapping = {
    '1s': (9, 9),
    '2s': (9, 9),
    '3s': (8, 8),
    '4s': (7, 6),
    '1d': (10, 2),
    '2d': (10, 3),
    '3d': (9, 3),
    '4d': (9, 5)
}
position_keys = list(positions_mapping.keys())

scores = []
predictions = {}
for path in wheel_paths:
    wheel_tensor = load_wheel_image(path).unsqueeze(0).to(device)
    with torch.no_grad():
        prediction = model(wheel_tensor).cpu().numpy()
        predictions[path] = prediction.flatten().tolist()
wheel_names = list(predictions.keys())
NUM_WHEELS = len(wheel_names)

# --- Pre-calcolo punteggi flip/non-flip ---
# precalc_scores[wheel][pos] = (score_normale, score_flippata)
precalc_scores = {}
for wheel in wheel_names:
    wheel_left, wheel_right = predictions[wheel]
    precalc_scores[wheel] = {}
    for pos in position_keys:
        weight_left, weight_right = positions_mapping[pos]
        score_normale = wheel_left * weight_left + wheel_right * weight_right
        score_flippata = wheel_right * weight_left + wheel_left * weight_right
        precalc_scores[wheel][pos] = (score_normale, score_flippata)

# --- Variabili per la ricerca ---
best_score = float("inf")  # vogliamo minimizzare
best_assignment = None

# ============================================================
# Funzione ricorsiva con branch-and-bound
# ============================================================

def search(assigned_idx=0, current_perm=[], current_score=0):
    global best_score, best_assignment

    if assigned_idx == NUM_WHEELS:
        # Fine ricorsione: tutte le posizioni assegnate
        if current_score < best_score:
            best_score = current_score
            best_assignment = current_perm.copy()
        return

    pos = position_keys[assigned_idx]

    # Proviamo tutte le ruote non ancora assegnate
    for wheel in wheel_names:
        if wheel in [w for w, _ in current_perm]:
            continue  # già assegnata

        score_normale, score_flippata = precalc_scores[wheel][pos]

        # --- Prova orientamento normale ---
        total_score = current_score + score_normale
        if total_score < best_score:  # Branch and bound
            current_perm.append((wheel, 0))  # 0 = normale
            search(assigned_idx + 1, current_perm, total_score)
            current_perm.pop()

        # --- Prova orientamento flippato ---
        total_score = current_score + score_flippata
        if total_score < best_score:
            current_perm.append((wheel, 1))  # 1 = flippata
            search(assigned_idx + 1, current_perm, total_score)
            current_perm.pop()

# ============================================================
# Avvio ricerca
# ============================================================

search()

# ============================================================
# Stampa risultato
# ============================================================

print("\n===============================================")
print(f"BEST SCORE: {best_score:.4f}")
print("MIGLIOR CONFIGURAZIONE TROVATA:")
for i, (wheel, flip) in enumerate(best_assignment):
    pos = position_keys[i]
    print(f"{pos} -> {wheel}   {'FLIPPATA' if flip else 'NORMALE'}")

In [ ]:
import itertools

# --- Variabili ---
best_score = float("inf")
best_assignment = None
total_combinations = 0

# wheel_names = ["ruota1","ruota2",...,"ruota8"]
# position_keys = ["1s", "2s", "3s", "4s", "1d", "2d", "3d", "4d"]
# precalc_scores = { wheel: {pos: (score_normale, score_flippata), ... }, ... }
NUM_WHEELS = len(position_keys)

# --- Genera tutte le permutazioni di ruote ---
for wheel_perm in itertools.permutations(wheel_names, NUM_WHEELS):
    # --- Genera tutte le combinazioni di orientamento (0=normale, 1=flippata) ---
    for orientation_combo in itertools.product([0,1], repeat=NUM_WHEELS):
        total_combinations += 1
        current_score = 0
        current_assignment = []

        # Calcola il punteggio totale per questa combinazione
        for idx, wheel in enumerate(wheel_perm):
            pos = position_keys[idx]
            orient = orientation_combo[idx]
            score_normale, score_flippata = precalc_scores[wheel][pos]
            score = float(score_normale) if orient == 0 else float(score_flippata)
            current_score += score
            current_assignment.append((wheel, orient))

        # Aggiorna il miglior punteggio
        if current_score < best_score:
            best_score = current_score
            best_assignment = current_assignment.copy()

print("Totale combinazioni testate:", total_combinations)
print("Miglior punteggio:", best_score)
print("Migliore assegnazione:", best_assignment)

# ============================================================
# Stampa risultato
# ============================================================

print("\n===============================================")
print(f"BEST SCORE: {best_score:.4f}")
print("MIGLIOR CONFIGURAZIONE TROVATA:")
for i, (wheel, flip) in enumerate(best_assignment):
    pos = position_keys[i]
    print(f"{pos} -> {wheel}   {'FLIPPATA' if flip else 'NORMALE'}")

In [ ]:
print(f"BEST SCORE: {best_score:.4f}")
print("MIGLIOR CONFIGURAZIONE TROVATA:")
for i, (wheel, flip) in enumerate(best_assignment):
    pos = position_keys[i]
    score_normale, score_flippata = precalc_scores[wheel][pos]
    coeff = score_flippata if flip else score_normale
    print(f"{pos} -> {wheel}   {'FLIPPATA' if flip else 'NORMALE'}   coeff: {predictions[wheel]}")